In [1]:
import numpy as np
import sympy as sp
from scipy.fft import fft
from scipy.integrate import quad
from sympy import symbols, diff, integrate, Matrix, laplace_transform


In [2]:
class CellEngine:
    def __init__(self):
        self.cache = {}

    def compute(self, name, func):
        if name not in self.cache:
            self.cache[name] = func()
        return self.cache[name]


In [3]:
class ALU:
    @staticmethod
    def add(a, b): return a + b
    @staticmethod
    def sub(a, b): return a - b
    @staticmethod
    def mul(a, b): return a * b
    @staticmethod
    def div(a, b): return a / b


In [4]:
def linear_algebra_cells(x, y):
    engine = CellEngine()
    x = np.array(x)
    y = np.array(y)

    results = {
        "dot_product": engine.compute("dot", lambda: np.dot(x, y)),
        "norm_x": engine.compute("norm_x", lambda: np.linalg.norm(x)),
        "norm_y": engine.compute("norm_y", lambda: np.linalg.norm(y)),
        "cos_theta": engine.compute(
            "cos",
            lambda: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
        )
    }
    return results


In [5]:
def calculus_cells(expr, var):
    engine = CellEngine()
    x = symbols(var)

    results = {
        "expression": expr,
        "derivative": engine.compute(
            "derivative", lambda: diff(expr, x)
        ),
        "integral": engine.compute(
            "integral", lambda: integrate(expr, x)
        )
    }
    return results


In [6]:
def fft_cells(signal):
    engine = CellEngine()
    signal = np.array(signal)

    results = {
        "original_signal": signal,
        "fft_result": engine.compute("fft", lambda: fft(signal)),
        "magnitude": engine.compute(
            "magnitude", lambda: np.abs(fft(signal))
        )
    }
    return results


In [7]:
def laplace_cells(expr, var='t'):
    engine = CellEngine()
    t, s = symbols(f"{var} s")

    results = {
        "expression": expr,
        "laplace": engine.compute(
            "laplace",
            lambda: laplace_transform(expr, t, s, noconds=True)
        )
    }
    return results


In [8]:
def vector_calculus_cells(F):
    engine = CellEngine()
    x, y, z = symbols('x y z')

    Fx, Fy, Fz = F
    divergence = diff(Fx, x) + diff(Fy, y) + diff(Fz, z)

    results = {
        "divergence": engine.compute("div", lambda: divergence),
        "curl": engine.compute(
            "curl",
            lambda: Matrix([
                diff(Fz, y) - diff(Fy, z),
                diff(Fx, z) - diff(Fz, x),
                diff(Fy, x) - diff(Fx, y)
            ])
        )
    }
    return results


In [9]:
def get_user_input():
    print("\nSelect operation:")
    print("1 - Arithmetic")
    print("2 - Linear Algebra")
    print("3 - Calculus")
    print("4 - FFT")
    print("5 - Laplace Transform")
    print("6 - Vector Calculus (Maxwell basics)")
    return input("Enter choice (1-6): ").strip()


In [10]:
def parse_vector(prompt):
    raw = input(prompt)
    return list(map(float, raw.split(',')))

def parse_expression(prompt):
    expr = input(prompt)
    return sp.sympify(expr)


In [11]:
def arithmetic_ui():
    a = float(input("Enter first number: "))
    b = float(input("Enter second number: "))
    op = input("Operation (+, -, *, /): ").strip()

    alu = ALU()
    operations = {
        '+': alu.add,
        '-': alu.sub,
        '*': alu.mul,
        '/': alu.div
    }

    if op not in operations:
        raise ValueError("Invalid arithmetic operation")

    return {"result": operations[op](a, b)}


In [12]:
def linear_algebra_ui():
    x = parse_vector("Enter vector x (comma-separated): ")
    y = parse_vector("Enter vector y (comma-separated): ")
    return linear_algebra_cells(x, y)


In [13]:
def calculus_ui():
    expr = parse_expression("Enter expression in x (e.g., x**2 + 3*x): ")
    return calculus_cells(expr, 'x')


In [14]:
def fft_ui():
    signal = parse_vector("Enter signal values (comma-separated): ")
    return fft_cells(signal)


In [15]:
def laplace_ui():
    expr = parse_expression("Enter function of t (e.g., sin(t)): ")
    return laplace_cells(expr, 't')


In [16]:
def vector_calculus_ui():
    print("Enter vector field F = (Fx, Fy, Fz)")
    Fx = parse_expression("Fx: ")
    Fy = parse_expression("Fy: ")
    Fz = parse_expression("Fz: ")
    return vector_calculus_cells((Fx, Fy, Fz))


In [17]:
def main():
    choice = get_user_input()

    dispatch = {
        '1': arithmetic_ui,
        '2': linear_algebra_ui,
        '3': calculus_ui,
        '4': fft_ui,
        '5': laplace_ui,
        '6': vector_calculus_ui
    }

    if choice not in dispatch:
        raise ValueError("Invalid choice")

    results = dispatch[choice]()
    print("\n--- Cell-wise Output ---")
    for cell, value in results.items():
        print(f"{cell}: {value}")


In [19]:
if __name__ == "__main__":
    main()



Select operation:
1 - Arithmetic
2 - Linear Algebra
3 - Calculus
4 - FFT
5 - Laplace Transform
6 - Vector Calculus (Maxwell basics)
Enter choice (1-6): 3
Enter expression in x (e.g., x**2 + 3*x): x**2

--- Cell-wise Output ---
expression: x**2
derivative: 2*x
integral: x**3/3
